In [3]:
import requests
import pandas as pd
import time

In [10]:
# define a function that can get 100 posts from a provided subreddit using pushshift API.
# Please note, 100 posts were the maximum number of post can be obained form reddit each time at the time when the function was created.
def get100_from_reddit(subreddit = 'StarWars', before = None):
#      input:  subreddit -> a string for the subreddit name
#               before - > the time stamp tells when the post was created, in the format of unix time (https://en.wikipedia.org/wiki/Unix_time)
#                            If it is given, this function only tries to get the post created before this time
    url = 'https://api.pushshift.io/reddit/search/submission'
    params = {
    'subreddit' : subreddit,
    'size' : 100,
    'before' : before
    }
    res = requests.get(url,params)
    if res.status_code == 200:
        data = res.json()
        posts = data['data']
    else:
        print(f"request status with error code: {res.status_code}")   # print out the the error information
    return pd.DataFrame(posts) 

In [48]:
df = get100_from_reddit()

In [55]:
df['created_utc']

0     1597808887
1     1597808693
2     1597808325
3     1597808200
4     1597808014
         ...    
95    1597781950
96    1597781870
97    1597781475
98    1597781320
99    1597781030
Name: created_utc, Length: 100, dtype: int64

In [11]:
# define a function that get the target amount of posts from the provided subreddit from reddit.com
def get_from_reddit(subreddit = 'StarWars', num_rows = 500):
#     input: subreddit -> a string for the subreddit name
#              num_rows -> The number of posts needed
    
#     No restrictions on the first 100 posts
    before = None 
    
#     The dataframe for the obtained posts
    df = get100_from_reddit(subreddit = subreddit)
    
    for i in range(200,num_rows+100,100):
#         Put time restriction on the posts by the oldest post which had been obained already
        before = df.iloc[df.shape[0]-1,:]['created_utc']
#     attached the newly obtained to the end of the dataframe which had all the posts we have already got
        df = pd.concat([df,get100_from_reddit(subreddit = subreddit,before=before)],axis = 0)
        
#         Printing out a message after getting every 1000 posts
        if i%1000 == 0:
            print(f"Have got {i} rows of data ")
#         Take 1 second break after getting 100 post each time
        time.sleep(1)
#     Return a dataframe which had all posts
    return df

In [24]:
# Obtain 20,000K posts from 'StarWars'(default subreddit) subreddit
df = get_from_reddit(num_rows = 20000)

Have got 1000 rows of data 
Have got 2000 rows of data 
Have got 3000 rows of data 
Have got 4000 rows of data 
Have got 5000 rows of data 
Have got 6000 rows of data 
Have got 7000 rows of data 
Have got 8000 rows of data 
Have got 9000 rows of data 
Have got 10000 rows of data 
Have got 11000 rows of data 
Have got 12000 rows of data 
Have got 13000 rows of data 
Have got 14000 rows of data 
Have got 15000 rows of data 
Have got 16000 rows of data 
Have got 17000 rows of data 
Have got 18000 rows of data 
Have got 19000 rows of data 
Have got 20000 rows of data 


In [25]:
# Obtain 20,000K posts from 'marvelstudios' subreddit
df_marvel = get_from_reddit(subreddit='marvelstudios', num_rows = 20000)

Have got 1000 rows of data 
Have got 2000 rows of data 
Have got 3000 rows of data 
Have got 4000 rows of data 
Have got 5000 rows of data 
Have got 6000 rows of data 
Have got 7000 rows of data 
Have got 8000 rows of data 
Have got 9000 rows of data 
Have got 10000 rows of data 
Have got 11000 rows of data 
Have got 12000 rows of data 
Have got 13000 rows of data 
Have got 14000 rows of data 
Have got 15000 rows of data 
Have got 16000 rows of data 
Have got 17000 rows of data 
Have got 18000 rows of data 
Have got 19000 rows of data 
Have got 20000 rows of data 


In [28]:
# Save the obtained datasets
df.to_csv('./data/starwar_0820.csv')
df_marvel.to_csv('./data/marvel_0820.csv')